# * VINSIGHT : Special Metric

## Parameter

In [191]:
import configparser
import datetime as dt
import pandas as pd
import numpy as np
import xlrd
import oracledb
import re
import FN_Monitoring as fn

config = configparser.ConfigParser()
config.read('../../my_config.ini')
config.sections()

TDMDBPR_user = config['TDMDBPR']['username']
TDMDBPR_pwd = config['TDMDBPR']['password']
TDMDBPR_db = config['TDMDBPR']['db']
TDMDBPR_host = config['TDMDBPR']['host']
TDMDBPR_port = config['TDMDBPR']['port']

AKPIPRD_user = config['AKPIPRD']['username']
AKPIPRD_pwd = config['AKPIPRD']['password']
AKPIPRD_db = config['AKPIPRD']['db']
AKPIPRD_host = config['AKPIPRD']['host']
AKPIPRD_port = config['AKPIPRD']['port']

curr_dt = dt.datetime.now().date()
str_curr_dt = curr_dt.strftime('%Y%m%d')

## Revenue Corporate (C, P)

In [192]:
''' Execute transaction '''


# Input parameter
curr_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
v_start_date = 20250101
print(f'\nParameter input...\n')
print(f'   -> Data as of {curr_datetime}')
print(f'   -> v_start_date: {v_start_date}')


# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
src_cur = src_conn.cursor()
query = (f"""
    SELECT /*+PARALLEL(8)*/
        TM_KEY_MTH, TM_KEY_DAY, PRODUCT_GRP, COMP_CD, METRIC_CD, METRIC_NAME --, AREA_TYPE, AREA_CD, AREA_NAME
        , MAX(ACTUAL_AS_OF) ACTUAL_AS_OF
        , SUM(CASE WHEN AREA_TYPE = 'C' THEN ACTUAL_SNAP END) C
        , SUM(CASE WHEN AREA_TYPE = 'P' THEN ACTUAL_SNAP END) P
        , SUM(CASE WHEN AREA_TYPE = 'G' THEN ACTUAL_SNAP END) G
        , SUM(CASE WHEN AREA_TYPE = 'H' THEN ACTUAL_SNAP END) H
        , SUM(CASE WHEN AREA_TYPE = 'HH' THEN ACTUAL_SNAP END) HH
        , MAX(PPN_TM) PPN_TM
    FROM GEOSPCAPPO.AGG_PERF_NEWCO
    WHERE METRIC_CD IN (
        'B0R000100' --Total Revenue
        , 'DB0R000100' --Total Revenue : DTAC
        , 'TB0R000100' --Total Revenue : TRUE
        , 'B0R000100GEO' --Total Revenue (Geo)
        , 'DB0R000100GEO' --Total Revenue (Geo) : DTAC
        , 'TB0R000100GEO' --Total Revenue (Geo) : TRUE

        , 'B0R000101' --Mobile Revenue
        , 'DB0R000101' --Mobile Revenue : DTAC
        , 'TB0R000101' --Mobile Revenue : TMH
        , 'B0R000101GEO' --Mobile Revenue (Geo)
        , 'DB0R000101GEO' --Mobile Revenue (Geo) : DTAC
        , 'TB0R000101GEO' --Mobile Revenue (Geo) : TMH
        
        , 'B2R000100' --Postpaid Revenue
        , 'B2R010100' --Postpaid Revenue B2C
        , 'B2R020100' --Postpaid Revenue B2B
        , 'DB2R000100' --Postpaid Revenue : DTAC
        , 'DB2R010100' --Postpaid Revenue B2C : DTAC
        , 'DB2R020100' --Postpaid Revenue B2B : DTAC
        , 'TB2R000100' --Postpaid Revenue : TMH
        , 'TB2R010100' --Postpaid Revenue B2C : TMH
        , 'TB2R020100' --Postpaid Revenue B2B : TMH
        
        , 'B1R000100' --Prepaid Revenue
        , 'DB1R000100' --Prepaid Revenue : DTAC
        , 'TB1R000100' --Prepaid Revenue : TMH
        
        , 'TB3R000100' --TOL Revenue
        
        , 'TB4R000100' --TVS Revenue
        )
    AND AREA_TYPE IN ('C','P','G','H','HH')
    AND TM_KEY_DAY >= {v_start_date}
    GROUP BY TM_KEY_MTH, TM_KEY_DAY, PRODUCT_GRP, COMP_CD, METRIC_CD, METRIC_NAME
""")


try:
    # Get : Timestamp
    src_cur.execute(query)
    rows = src_cur.fetchall()
    chk_src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])

    src_cur.close()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()


Parameter input...

   -> Data as of 2025-05-26, 18:26:57
   -> v_start_date: 20250101


In [193]:
''' Automate Month Period '''

curr_mth = chk_src_df['TM_KEY_MTH'].max()
prev_mth = str(chk_src_df['TM_KEY_MTH'].drop_duplicates().astype(int).shift().max())

In [194]:
''' Monthly Summary '''

monthly_df = chk_src_df.copy()
monthly_df = monthly_df.groupby(['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME']).agg({'PPN_TM':'max', 'C':'sum', 'P':'sum', 'G':'sum', 'H':'sum', 'HH':'sum'})
monthly_df = monthly_df.fillna(0).sort_values(by=['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD']).reset_index()
monthly_df['DIFF (C-P)'] = monthly_df['C'] - monthly_df['P']

monthly_df_display = monthly_df.copy()
mod_col_list = monthly_df_display.iloc[:, 5:].columns.tolist()
for col in mod_col_list:
    monthly_df_display[col] = monthly_df_display[col].apply(lambda x: format(x, ',.0f'))

# monthly_df_display

### Sample Data

In [195]:
sample_mth_df = monthly_df_display.loc[monthly_df_display['TM_KEY_MTH']==202501]
sample_mth_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH,DIFF (C-P)
0,202501,All Services,B0R000100,Total Revenue,2025-05-26 00:09:03,"11,798,929,827","12,636,945,503",0,0,0,"-838,015,676"
1,202501,All Services,B0R000100GEO,Total Revenue (Geo),2025-05-26 03:06:46,0,"12,034,347,430","12,018,244,403","12,017,986,832","12,017,979,514","-12,034,347,430"
2,202501,All Services,DB0R000100,Total Revenue : DTAC,2025-05-26 03:06:46,"7,339,630,516","4,579,941,567","4,579,940,849","4,577,983,551","4,577,983,551","2,759,688,949"
3,202501,All Services,DB0R000100GEO,Total Revenue (Geo) : DTAC,2025-05-26 03:06:46,0,"4,371,652,537","4,371,651,819","4,371,510,313","4,371,510,313","-4,371,652,537"
4,202501,All Services,TB0R000100,Total Revenue : TRUE,2025-05-26 03:06:46,"4,459,299,311","8,057,003,936","4,316,603,800","4,316,492,052","4,316,484,734","-3,597,704,625"
5,202501,All Services,TB0R000100GEO,Total Revenue (Geo) : TRUE,2025-05-26 03:06:46,0,"7,662,694,892","7,646,592,584","7,646,476,519","7,646,469,201","-7,662,694,892"
6,202501,Mobile,B0R000101,Mobile Revenue,2025-05-26 00:09:03,"9,784,556,756","10,751,551,183",0,0,0,"-966,994,427"
7,202501,Mobile,B0R000101GEO,Mobile Revenue (Geo),2025-05-26 03:06:46,0,"10,148,953,110","10,132,850,084","10,132,690,811","10,132,690,811","-10,148,953,110"
8,202501,Mobile,DB0R000101,Mobile Revenue : DTAC,2025-05-26 00:09:03,"7,339,630,516","4,579,941,567",0,0,0,"2,759,688,949"
9,202501,Mobile,DB0R000101GEO,Mobile Revenue (Geo) : DTAC,2025-05-26 03:06:46,0,"4,371,652,537","4,371,651,819","4,371,510,313","4,371,510,313","-4,371,652,537"


### Peparing Check Sum Data

In [196]:
''' B0R000100 : Total Revenue '''

v_metric_cd = 'B0R000100'
v_metric_name = 'Total Revenue (sum)'
v_metric_list = [
    'DB1R000100' # Prepaid Revenue : DTAC
    , 'TB1R000100' # Prepaid Revenue : TMH
    , 'DB2R000100' # Postpaid Revenue : DTAC
    , 'TB2R000100' # Postpaid Revenue : TMH
    , 'TB3R000100' # TOL Revenue
    , 'TB4R000100' # TVS Revenue
    ]

chk_B0R000100_df = monthly_df.copy()
chk_B0R000100_df = chk_B0R000100_df.loc[chk_B0R000100_df['METRIC_CD'].isin(v_metric_list)]
chk_B0R000100_df['METRIC_CD'] = v_metric_cd
chk_B0R000100_df['METRIC_NAME'] = v_metric_name
chk_B0R000100_df = chk_B0R000100_df.groupby(['TM_KEY_MTH', 'METRIC_CD', 'METRIC_NAME']).agg({'C':'sum', 'P':'sum'})
chk_B0R000100_df.rename(columns={'C':'C (Sum)', 'P':'P (Sum)'}, inplace=True)
chk_B0R000100_df = chk_B0R000100_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()

chk_B0R000100_df_display = chk_B0R000100_df.copy()
mod_col_list = chk_B0R000100_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    chk_B0R000100_df_display[col] = chk_B0R000100_df_display[col].apply(lambda x: format(x, ',.0f'))
chk_B0R000100_df_display

,TM_KEY_MTH,METRIC_CD,METRIC_NAME,C (Sum),P (Sum)
0,202501,B0R000100,Total Revenue (sum),"12,770,106,773","12,636,945,503"
1,202502,B0R000100,Total Revenue (sum),"12,236,601,216","12,212,436,891"
2,202503,B0R000100,Total Revenue (sum),"12,516,306,281","12,587,616,315"
3,202504,B0R000100,Total Revenue (sum),"12,526,105,797","12,501,905,390"
4,202505,B0R000100,Total Revenue (sum),"9,660,502,891","9,634,662,936"


In [197]:
''' TB0R000100 : Total Revenue : TMH '''

v_metric_cd = 'TB0R000100'
v_metric_name = 'Total Revenue : TMH (sum)'
v_metric_list = [
    'TB1R000100' # Prepaid Revenue : TMH
    , 'TB2R000100' # Postpaid Revenue : TMH
    , 'TB3R000100' # TOL Revenue
    , 'TB4R000100' # TVS Revenue
    ]

chk_TB0R000100_df = monthly_df.copy()
chk_TB0R000100_df = chk_TB0R000100_df.loc[chk_TB0R000100_df['METRIC_CD'].isin(v_metric_list)]
chk_TB0R000100_df['METRIC_CD'] = v_metric_cd
chk_TB0R000100_df['METRIC_NAME'] = v_metric_name
chk_TB0R000100_df = chk_TB0R000100_df.groupby(['TM_KEY_MTH', 'METRIC_CD', 'METRIC_NAME']).agg({'C':'sum', 'P':'sum'})
chk_TB0R000100_df.rename(columns={'C':'C (Sum)', 'P':'P (Sum)'}, inplace=True)
chk_TB0R000100_df = chk_TB0R000100_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()

chk_TB0R000100_df_display = chk_TB0R000100_df.copy()
mod_col_list = chk_TB0R000100_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    chk_TB0R000100_df_display[col] = chk_TB0R000100_df_display[col].apply(lambda x: format(x, ',.0f'))
chk_TB0R000100_df_display

,TM_KEY_MTH,METRIC_CD,METRIC_NAME,C (Sum),P (Sum)
0,202501,TB0R000100,Total Revenue : TMH (sum),"8,190,378,845","8,057,003,936"
1,202502,TB0R000100,Total Revenue : TMH (sum),"7,860,591,688","7,836,255,273"
2,202503,TB0R000100,Total Revenue : TMH (sum),"8,005,985,591","8,077,214,145"
3,202504,TB0R000100,Total Revenue : TMH (sum),"8,067,988,282","8,043,775,917"
4,202505,TB0R000100,Total Revenue : TMH (sum),"6,202,764,974","6,176,920,305"


In [198]:
''' DB0R000100 : Total Revenue : DTAC '''

v_metric_cd = 'DB0R000100'
v_metric_name = 'Total Revenue : DTAC (sum)'
v_metric_list = [
    'DB1R000100' # Prepaid Revenue : DTAC
    , 'DB2R000100' # Postpaid Revenue : DTAC
    ]

chk_DB0R000100_df = monthly_df.copy()
chk_DB0R000100_df = chk_DB0R000100_df.loc[chk_DB0R000100_df['METRIC_CD'].isin(v_metric_list)]
chk_DB0R000100_df['METRIC_CD'] = v_metric_cd
chk_DB0R000100_df['METRIC_NAME'] = v_metric_name
chk_DB0R000100_df = chk_DB0R000100_df.groupby(['TM_KEY_MTH', 'METRIC_CD', 'METRIC_NAME']).agg({'C':'sum', 'P':'sum'})
chk_DB0R000100_df.rename(columns={'C':'C (Sum)', 'P':'P (Sum)'}, inplace=True)
chk_DB0R000100_df = chk_DB0R000100_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()

chk_DB0R000100_df_display = chk_DB0R000100_df.copy()
mod_col_list = chk_DB0R000100_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    chk_DB0R000100_df_display[col] = chk_DB0R000100_df_display[col].apply(lambda x: format(x, ',.0f'))
chk_DB0R000100_df_display

,TM_KEY_MTH,METRIC_CD,METRIC_NAME,C (Sum),P (Sum)
0,202501,DB0R000100,Total Revenue : DTAC (sum),"4,579,727,928","4,579,941,567"
1,202502,DB0R000100,Total Revenue : DTAC (sum),"4,376,009,529","4,376,181,618"
2,202503,DB0R000100,Total Revenue : DTAC (sum),"4,510,320,691","4,510,402,170"
3,202504,DB0R000100,Total Revenue : DTAC (sum),"4,458,117,515","4,458,129,472"
4,202505,DB0R000100,Total Revenue : DTAC (sum),"3,457,737,917","3,457,742,631"


In [199]:
''' B0R000101 : Mobile Revenue '''

v_metric_cd = 'B0R000101'
v_metric_name = 'Mobile Revenue (sum)'
v_metric_list = [
    'DB1R000100' # Prepaid Revenue : DTAC
    , 'TB1R000100' # Prepaid Revenue : TMH
    , 'DB2R000100' # Postpaid Revenue : DTAC
    , 'TB2R000100' # Postpaid Revenue : TMH
    ]

chk_B0R000101_df = monthly_df.copy()
chk_B0R000101_df = chk_B0R000101_df.loc[chk_B0R000101_df['METRIC_CD'].isin(v_metric_list)]
chk_B0R000101_df['METRIC_CD'] = v_metric_cd
chk_B0R000101_df['METRIC_NAME'] = v_metric_name
chk_B0R000101_df = chk_B0R000101_df.groupby(['TM_KEY_MTH', 'METRIC_CD', 'METRIC_NAME']).agg({'C':'sum', 'P':'sum'})
chk_B0R000101_df.rename(columns={'C':'C (Sum)', 'P':'P (Sum)'}, inplace=True)
chk_B0R000101_df = chk_B0R000101_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()

chk_B0R000101_df_display = chk_B0R000101_df.copy()
mod_col_list = chk_B0R000101_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    chk_B0R000101_df_display[col] = chk_B0R000101_df_display[col].apply(lambda x: format(x, ',.0f'))
chk_B0R000101_df_display

,TM_KEY_MTH,METRIC_CD,METRIC_NAME,C (Sum),P (Sum)
0,202501,B0R000101,Mobile Revenue (sum),"10,755,733,702","10,751,551,183"
1,202502,B0R000101,Mobile Revenue (sum),"10,319,699,474","10,314,001,696"
2,202503,B0R000101,Mobile Revenue (sum),"10,685,245,593","10,678,120,361"
3,202504,B0R000101,Mobile Revenue (sum),"10,592,109,386","10,585,358,700"
4,202505,B0R000101,Mobile Revenue (sum),"8,221,478,575","8,208,476,324"


In [200]:
''' TB0R000101 : Mobile Revenue : TMH '''

v_metric_cd = 'TB0R000101'
v_metric_name = 'Mobile Revenue : TMH (sum)'
v_metric_list = [
    'TB1R000100' # Prepaid Revenue : TMH
    , 'TB2R000100' # Postpaid Revenue : TMH
    ]

chk_TB0R000101_df = monthly_df.copy()
chk_TB0R000101_df = chk_TB0R000101_df.loc[chk_TB0R000101_df['METRIC_CD'].isin(v_metric_list)]
chk_TB0R000101_df['METRIC_CD'] = v_metric_cd
chk_TB0R000101_df['METRIC_NAME'] = v_metric_name
chk_TB0R000101_df = chk_TB0R000101_df.groupby(['TM_KEY_MTH', 'METRIC_CD', 'METRIC_NAME']).agg({'C':'sum', 'P':'sum'})
chk_TB0R000101_df.rename(columns={'C':'C (Sum)', 'P':'P (Sum)'}, inplace=True)
chk_TB0R000101_df = chk_TB0R000101_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()

chk_TB0R000101_df_display = chk_TB0R000101_df.copy()
mod_col_list = chk_TB0R000101_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    chk_TB0R000101_df_display[col] = chk_TB0R000101_df_display[col].apply(lambda x: format(x, ',.0f'))
chk_TB0R000101_df_display

,TM_KEY_MTH,METRIC_CD,METRIC_NAME,C (Sum),P (Sum)
0,202501,TB0R000101,Mobile Revenue : TMH (sum),"6,176,005,774","6,171,609,616"
1,202502,TB0R000101,Mobile Revenue : TMH (sum),"5,943,689,946","5,937,820,079"
2,202503,TB0R000101,Mobile Revenue : TMH (sum),"6,174,924,902","6,167,718,192"
3,202504,TB0R000101,Mobile Revenue : TMH (sum),"6,133,991,871","6,127,229,228"
4,202505,TB0R000101,Mobile Revenue : TMH (sum),"4,763,740,658","4,750,733,693"


In [201]:
''' DB0R000101 : Mobile Revenue : DTAC '''

v_metric_cd = 'DB0R000101'
v_metric_name = 'Mobile Revenue : DTAC (sum)'
v_metric_list = [
    'DB1R000100' # Prepaid Revenue : DTAC
    , 'DB2R000100' # Postpaid Revenue : DTAC
    ]

chk_DB0R000101_df = monthly_df.copy()
chk_DB0R000101_df = chk_DB0R000101_df.loc[chk_DB0R000101_df['METRIC_CD'].isin(v_metric_list)]
chk_DB0R000101_df['METRIC_CD'] = v_metric_cd
chk_DB0R000101_df['METRIC_NAME'] = v_metric_name
chk_DB0R000101_df = chk_DB0R000101_df.groupby(['TM_KEY_MTH', 'METRIC_CD', 'METRIC_NAME']).agg({'C':'sum', 'P':'sum'})
chk_DB0R000101_df.rename(columns={'C':'C (Sum)', 'P':'P (Sum)'}, inplace=True)
chk_DB0R000101_df = chk_DB0R000101_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()

chk_DB0R000101_df_display = chk_DB0R000101_df.copy()
mod_col_list = chk_DB0R000101_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    chk_DB0R000101_df_display[col] = chk_DB0R000101_df_display[col].apply(lambda x: format(x, ',.0f'))
chk_DB0R000101_df_display

,TM_KEY_MTH,METRIC_CD,METRIC_NAME,C (Sum),P (Sum)
0,202501,DB0R000101,Mobile Revenue : DTAC (sum),"4,579,727,928","4,579,941,567"
1,202502,DB0R000101,Mobile Revenue : DTAC (sum),"4,376,009,529","4,376,181,618"
2,202503,DB0R000101,Mobile Revenue : DTAC (sum),"4,510,320,691","4,510,402,170"
3,202504,DB0R000101,Mobile Revenue : DTAC (sum),"4,458,117,515","4,458,129,472"
4,202505,DB0R000101,Mobile Revenue : DTAC (sum),"3,457,737,917","3,457,742,631"


In [202]:
''' Concat All Check Sum Data '''

sum_rev_corp_df = pd.concat([chk_B0R000100_df, chk_TB0R000100_df, chk_DB0R000100_df, chk_B0R000101_df, chk_TB0R000101_df, chk_DB0R000101_df])
sum_rev_corp_df

,TM_KEY_MTH,METRIC_CD,METRIC_NAME,C (Sum),P (Sum)
0,202501,B0R000100,Total Revenue (sum),1.277011e+10,1.263695e+10
1,202502,B0R000100,Total Revenue (sum),1.223660e+10,1.221244e+10
2,202503,B0R000100,Total Revenue (sum),1.251631e+10,1.258762e+10
3,202504,B0R000100,Total Revenue (sum),1.252611e+10,1.250191e+10
4,202505,B0R000100,Total Revenue (sum),9.660503e+09,9.634663e+09
0,202501,TB0R000100,Total Revenue : TMH (sum),8.190379e+09,8.057004e+09
1,202502,TB0R000100,Total Revenue : TMH (sum),7.860592e+09,7.836255e+09
2,202503,TB0R000100,Total Revenue : TMH (sum),8.005986e+09,8.077214e+09
3,202504,TB0R000100,Total Revenue : TMH (sum),8.067988e+09,8.043776e+09
4,202505,TB0R000100,Total Revenue : TMH (sum),6.202765e+09,6.176920e+09


### Comparison Overview

In [203]:
''' Total & Mobile '''

v_metric_list = [
    'B0R000100' # Total Revenue
    , 'DB0R000100' # Total Revenue : DTAC
    , 'TB0R000100' # Total Revenue : TRUE
    , 'B0R000101' # Mobile Revenue
    , 'DB0R000101' # Mobile Revenue : DTAC
    , 'TB0R000101' # Mobile Revenue : TMH
    ]

vis_rev_corp_df = monthly_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'C', 'P', 'DIFF (C-P)']].copy()
vis_rev_corp_df = vis_rev_corp_df.loc[vis_rev_corp_df['METRIC_CD'].isin(v_metric_list)]

merge_comparison_df = pd.merge(vis_rev_corp_df, sum_rev_corp_df[['TM_KEY_MTH', 'METRIC_CD', 'C (Sum)', 'P (Sum)']], how='left', on=['TM_KEY_MTH', 'METRIC_CD'])
merge_comparison_df['C (Diff)'] = merge_comparison_df['C'] - merge_comparison_df['C (Sum)']
merge_comparison_df['P (Diff)'] = merge_comparison_df['P'] - merge_comparison_df['P (Sum)']

comparison_df = merge_comparison_df.fillna(0).sort_values(by=['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD']).reset_index(drop=True)

comparison_df_display = comparison_df.copy()
mod_col_list = comparison_df_display.iloc[:, 5:].columns.tolist()
for col in mod_col_list:
    comparison_df_display[col] = comparison_df_display[col].apply(lambda x: format(x, ',.0f'))

comparison_df_display

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,DIFF (C-P),C (Sum),P (Sum),C (Diff),P (Diff)
0,202501,All Services,B0R000100,Total Revenue,2025-05-26 00:09:03,"11,798,929,827","12,636,945,503","-838,015,676","12,770,106,773","12,636,945,503","-971,176,946",0
1,202501,All Services,DB0R000100,Total Revenue : DTAC,2025-05-26 03:06:46,"7,339,630,516","4,579,941,567","2,759,688,949","4,579,727,928","4,579,941,567","2,759,902,588",0
2,202501,All Services,TB0R000100,Total Revenue : TRUE,2025-05-26 03:06:46,"4,459,299,311","8,057,003,936","-3,597,704,625","8,190,378,845","8,057,003,936","-3,731,079,534",0
3,202501,Mobile,B0R000101,Mobile Revenue,2025-05-26 00:09:03,"9,784,556,756","10,751,551,183","-966,994,427","10,755,733,702","10,751,551,183","-971,176,946",0
4,202501,Mobile,DB0R000101,Mobile Revenue : DTAC,2025-05-26 00:09:03,"7,339,630,516","4,579,941,567","2,759,688,949","4,579,727,928","4,579,941,567","2,759,902,588",0
5,202501,Mobile,TB0R000101,Mobile Revenue : TMH,2025-05-26 00:09:03,"2,444,926,240","6,171,609,616","-3,726,683,376","6,176,005,774","6,171,609,616","-3,731,079,534",0
6,202502,All Services,B0R000100,Total Revenue,2025-05-26 00:09:03,"10,910,887,639","12,212,436,891","-1,301,549,252","12,236,601,216","12,212,436,891","-1,325,713,577",0
7,202502,All Services,DB0R000100,Total Revenue : DTAC,2025-05-26 03:06:46,"6,776,726,890","4,376,181,618","2,400,545,272","4,376,009,529","4,376,181,618","2,400,717,361",0
8,202502,All Services,TB0R000100,Total Revenue : TRUE,2025-05-26 03:06:46,"4,134,160,749","7,836,255,273","-3,702,094,524","7,860,591,688","7,836,255,273","-3,726,430,939",0
9,202502,Mobile,B0R000101,Mobile Revenue,2025-05-26 00:09:03,"8,993,985,897","10,314,001,696","-1,320,015,799","10,319,699,474","10,314,001,696","-1,325,713,577",0


### Comparison by Month

In [204]:
y202501_comparison_df = comparison_df_display.loc[comparison_df_display['TM_KEY_MTH']==202501]
y202501_comparison_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,DIFF (C-P),C (Sum),P (Sum),C (Diff),P (Diff)
0,202501,All Services,B0R000100,Total Revenue,2025-05-26 00:09:03,"11,798,929,827","12,636,945,503","-838,015,676","12,770,106,773","12,636,945,503","-971,176,946",0
1,202501,All Services,DB0R000100,Total Revenue : DTAC,2025-05-26 03:06:46,"7,339,630,516","4,579,941,567","2,759,688,949","4,579,727,928","4,579,941,567","2,759,902,588",0
2,202501,All Services,TB0R000100,Total Revenue : TRUE,2025-05-26 03:06:46,"4,459,299,311","8,057,003,936","-3,597,704,625","8,190,378,845","8,057,003,936","-3,731,079,534",0
3,202501,Mobile,B0R000101,Mobile Revenue,2025-05-26 00:09:03,"9,784,556,756","10,751,551,183","-966,994,427","10,755,733,702","10,751,551,183","-971,176,946",0
4,202501,Mobile,DB0R000101,Mobile Revenue : DTAC,2025-05-26 00:09:03,"7,339,630,516","4,579,941,567","2,759,688,949","4,579,727,928","4,579,941,567","2,759,902,588",0
5,202501,Mobile,TB0R000101,Mobile Revenue : TMH,2025-05-26 00:09:03,"2,444,926,240","6,171,609,616","-3,726,683,376","6,176,005,774","6,171,609,616","-3,731,079,534",0


In [205]:
y202502_comparison_df = comparison_df_display.loc[comparison_df_display['TM_KEY_MTH']==202502]
y202502_comparison_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,DIFF (C-P),C (Sum),P (Sum),C (Diff),P (Diff)
6,202502,All Services,B0R000100,Total Revenue,2025-05-26 00:09:03,"10,910,887,639","12,212,436,891","-1,301,549,252","12,236,601,216","12,212,436,891","-1,325,713,577",0
7,202502,All Services,DB0R000100,Total Revenue : DTAC,2025-05-26 03:06:46,"6,776,726,890","4,376,181,618","2,400,545,272","4,376,009,529","4,376,181,618","2,400,717,361",0
8,202502,All Services,TB0R000100,Total Revenue : TRUE,2025-05-26 03:06:46,"4,134,160,749","7,836,255,273","-3,702,094,524","7,860,591,688","7,836,255,273","-3,726,430,939",0
9,202502,Mobile,B0R000101,Mobile Revenue,2025-05-26 00:09:03,"8,993,985,897","10,314,001,696","-1,320,015,799","10,319,699,474","10,314,001,696","-1,325,713,577",0
10,202502,Mobile,DB0R000101,Mobile Revenue : DTAC,2025-05-26 00:09:03,"6,776,726,890","4,376,181,618","2,400,545,272","4,376,009,529","4,376,181,618","2,400,717,361",0
11,202502,Mobile,TB0R000101,Mobile Revenue : TMH,2025-05-26 00:09:03,"2,217,259,007","5,937,820,079","-3,720,561,071","5,943,689,946","5,937,820,079","-3,726,430,939",0


In [206]:
y202503_comparison_df = comparison_df_display.loc[comparison_df_display['TM_KEY_MTH']==202503]
y202503_comparison_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,DIFF (C-P),C (Sum),P (Sum),C (Diff),P (Diff)
12,202503,All Services,B0R000100,Total Revenue,2025-05-26 00:09:03,"11,542,558,166","12,587,616,315","-1,045,058,149","12,516,306,281","12,587,616,315","-973,748,116",0
13,202503,All Services,DB0R000100,Total Revenue : DTAC,2025-05-26 03:06:46,"7,245,126,495","4,510,402,170","2,734,724,325","4,510,320,691","4,510,402,170","2,734,805,804",0
14,202503,All Services,TB0R000100,Total Revenue : TRUE,2025-05-26 03:06:46,"4,297,431,671","8,077,214,145","-3,779,782,474","8,005,985,591","8,077,214,145","-3,708,553,920",0
15,202503,Mobile,B0R000101,Mobile Revenue,2025-05-26 00:09:03,"9,711,497,477","10,678,120,361","-966,622,884","10,685,245,593","10,678,120,361","-973,748,116",0
16,202503,Mobile,DB0R000101,Mobile Revenue : DTAC,2025-05-26 00:09:03,"7,245,126,495","4,510,402,170","2,734,724,325","4,510,320,691","4,510,402,170","2,734,805,804",0
17,202503,Mobile,TB0R000101,Mobile Revenue : TMH,2025-05-26 00:09:03,"2,466,370,982","6,167,718,192","-3,701,347,210","6,174,924,902","6,167,718,192","-3,708,553,920",0


In [207]:
y202504_comparison_df = comparison_df_display.loc[comparison_df_display['TM_KEY_MTH']==202504]
y202504_comparison_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,DIFF (C-P),C (Sum),P (Sum),C (Diff),P (Diff)
18,202504,All Services,B0R000100,Total Revenue,2025-05-26 00:09:03,"8,758,472,043","12,501,905,390","-3,743,433,347","12,526,105,797","12,501,905,390","-3,767,633,755",0
19,202504,All Services,DB0R000100,Total Revenue : DTAC,2025-05-26 03:06:46,"4,458,117,515","4,458,129,472","-11,957","4,458,117,515","4,458,129,472",-0,0
20,202504,All Services,TB0R000100,Total Revenue : TRUE,2025-05-26 03:06:46,"4,300,354,528","8,043,775,917","-3,743,421,390","8,067,988,282","8,043,775,917","-3,767,633,755",0
21,202504,Mobile,B0R000101,Mobile Revenue,2025-05-26 00:09:03,"6,824,475,632","10,585,358,700","-3,760,883,069","10,592,109,386","10,585,358,700","-3,767,633,755",0
22,202504,Mobile,DB0R000101,Mobile Revenue : DTAC,2025-05-26 00:09:03,"4,458,117,515","4,458,129,472","-11,957","4,458,117,515","4,458,129,472",-0,0
23,202504,Mobile,TB0R000101,Mobile Revenue : TMH,2025-05-26 00:09:03,"2,366,358,116","6,127,229,228","-3,760,871,112","6,133,991,871","6,127,229,228","-3,767,633,755",0


In [208]:
y202505_comparison_df = comparison_df_display.loc[comparison_df_display['TM_KEY_MTH']==202505]
y202505_comparison_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,DIFF (C-P),C (Sum),P (Sum),C (Diff),P (Diff)
24,202505,All Services,B0R000100,Total Revenue,2025-05-26 00:09:03,"6,721,566,822","9,635,104,403","-2,913,537,581","9,660,502,891","9,634,662,936","-2,938,936,069","441,467"
25,202505,All Services,DB0R000100,Total Revenue : DTAC,2025-05-26 03:06:46,"3,457,737,917","3,457,742,631","-4,714","3,457,737,917","3,457,742,631",0,-0
26,202505,All Services,TB0R000100,Total Revenue : TRUE,2025-05-26 03:06:46,"3,263,828,906","6,177,361,773","-2,913,532,867","6,202,764,974","6,176,920,305","-2,938,936,069","441,467"
27,202505,Mobile,B0R000101,Mobile Revenue,2025-05-26 00:09:03,"5,282,542,506","8,208,917,791","-2,926,375,285","8,221,478,575","8,208,476,324","-2,938,936,069","441,467"
28,202505,Mobile,DB0R000101,Mobile Revenue : DTAC,2025-05-26 00:09:03,"3,457,737,917","3,457,742,631","-4,714","3,457,737,917","3,457,742,631",0,-0
29,202505,Mobile,TB0R000101,Mobile Revenue : TMH,2025-05-26 00:09:03,"1,824,804,589","4,751,175,160","-2,926,370,571","4,763,740,658","4,750,733,693","-2,938,936,069","441,467"


In [209]:
y202506_comparison_df = comparison_df_display.loc[comparison_df_display['TM_KEY_MTH']==202506]
y202506_comparison_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,DIFF (C-P),C (Sum),P (Sum),C (Diff),P (Diff)


### Pending...

In [210]:
# ''' B0R000100 : Total Revenue '''

# v_total_metric_list = [
#     'DB1R000100' # Prepaid Revenue : DTAC
#     , 'TB1R000100' # Prepaid Revenue : TMH
#     , 'DB2R000100' # Postpaid Revenue : DTAC
#     , 'TB2R000100' # Postpaid Revenue : TMH
#     , 'TB3R000100' # TOL Revenue
#     , 'TB4R000100' # TVS Revenue
#     ]

# B0R000100_day_df = chk_src_df.copy()
# B0R000100_day_df = B0R000100_day_df.loc[B0R000100_day_df['METRIC_CD'].isin(v_total_metric_list)]

# # B0R000100_day_df = chk_src_df.loc[chk_src_df['TM_KEY_MTH']==202501].copy()
# B0R000100_day_df['METRIC_TMP'] = B0R000100_day_df['METRIC_NAME'].replace(r' : DTAC| : TRUE| : TMH', '', regex=True)
# B0R000100_day_df['ALL'] = np.where(B0R000100_day_df['COMP_CD']=='ALL', B0R000100_day_df['C'], 0)
# B0R000100_day_df['TRUE'] = np.where(B0R000100_day_df['COMP_CD']=='TRUE', B0R000100_day_df['C'], 0)
# B0R000100_day_df['DTAC'] = np.where(B0R000100_day_df['COMP_CD']=='DTAC', B0R000100_day_df['C'], 0)

# B0R000100_day_df = B0R000100_day_df.groupby(['TM_KEY_MTH', 'METRIC_TMP']).agg({'PPN_TM':'max', 'ALL':'sum', 'TRUE':'sum', 'DTAC':'sum'})
# B0R000100_day_df = B0R000100_day_df.fillna(0).sort_values(by=['TM_KEY_MTH', 'METRIC_TMP']).reset_index()

# B0R000100_day_df['SUM (T+D)'] = B0R000100_day_df['TRUE'] + B0R000100_day_df['DTAC']
# B0R000100_day_df['DIFF (SUM-ALL)'] = B0R000100_day_df['SUM (T+D)'] - B0R000100_day_df['ALL']

# mod_col_list = B0R000100_day_df.iloc[:, 3:8].columns.tolist()
# for col in mod_col_list:
#     B0R000100_day_df[col] = B0R000100_day_df[col].apply(lambda x: format(x, ',.0f'))

# B0R000100_day_df